In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import datasets

iris=datasets.load_iris()
print(iris.data)
print(iris.target)
print(iris.data.shape)
print(iris.target.shape)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [50]:
x_train=torch.FloatTensor(iris.data)
y_train=torch.LongTensor(iris.target)
print(x_train.size())
print(y_train.size())

torch.Size([150, 4])
torch.Size([150])


In [51]:
y_one_hot=torch.zeros(150, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1),1)
print(y_train[145:])
print(y_one_hot[145:])

tensor([2, 2, 2, 2, 2])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])


In [52]:
w=torch.zeros((4,3), requires_grad=True)
b=torch.zeros((1,3), requires_grad=True)
optimizer=optim.SGD([w, b], lr=0.1)

In [53]:
for epoch in range(10001):
  y_hat=F.softmax(x_train.matmul(w)+b, dim=1)
  cost=(y_one_hot * -torch.log(y_hat)).sum(dim=1).mean()

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  if epoch%400==0:
    print('epoch:{}, cost:{:.4f}'.format(epoch, cost.item()))

epoch:0, cost:1.0986


epoch:400, cost:0.1915
epoch:800, cost:0.1389
epoch:1200, cost:0.1166
epoch:1600, cost:0.1040
epoch:2000, cost:0.0957
epoch:2400, cost:0.0899
epoch:2800, cost:0.0854
epoch:3200, cost:0.0820
epoch:3600, cost:0.0791
epoch:4000, cost:0.0768
epoch:4400, cost:0.0748
epoch:4800, cost:0.0731
epoch:5200, cost:0.0716
epoch:5600, cost:0.0703
epoch:6000, cost:0.0691
epoch:6400, cost:0.0681
epoch:6800, cost:0.0671
epoch:7200, cost:0.0662
epoch:7600, cost:0.0654
epoch:8000, cost:0.0647
epoch:8400, cost:0.0640
epoch:8800, cost:0.0634
epoch:9200, cost:0.0628
epoch:9600, cost:0.0623
epoch:10000, cost:0.0618


In [54]:
pred=F.softmax(x_train.matmul(w)+b, dim=1).argmax(dim=1)
accuracy=(pred==y_train).sum()/len(pred)*100
print(accuracy)

tensor(98.)


1. prtorch 기본 소스로 모델 작성
2. 클래스로 모델 작성

In [55]:
class SoftmaxClassModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(4,3)

  def forward(self, x):
    return self.linear(x)

In [56]:
model=SoftmaxClassModel()
optimizer=optim.SGD(model.parameters(), lr=0.1)
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.0768,  0.3266, -0.4918,  0.1034],
        [ 0.0320,  0.2459, -0.2637, -0.3364],
        [-0.4998,  0.0084, -0.2865, -0.0979]], requires_grad=True), Parameter containing:
tensor([-0.4901, -0.2619,  0.1359], requires_grad=True)]


In [57]:
for epoch in range(2001):
  y_hat=model(x_train)
  cost=F.cross_entropy(y_hat, y_train)
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  if epoch % 100 ==0:
    accuracy=(y_hat.argmax(dim=1)==y_train).sum()/len(y_hat)
    print('epoch:{}, cost:{:.5f}, accuracy:{:.5f}'.format(epoch,cost.item(), 
                                                          accuracy))

epoch:0, cost:1.75359, accuracy:0.33333
epoch:100, cost:0.42754, accuracy:0.77333
epoch:200, cost:0.26092, accuracy:0.96667
epoch:300, cost:0.21897, accuracy:0.98000
epoch:400, cost:0.19207, accuracy:0.98000
epoch:500, cost:0.17326, accuracy:0.98000
epoch:600, cost:0.15931, accuracy:0.98000
epoch:700, cost:0.14852, accuracy:0.98000
epoch:800, cost:0.13990, accuracy:0.98000
epoch:900, cost:0.13284, accuracy:0.98000
epoch:1000, cost:0.12694, accuracy:0.98000
epoch:1100, cost:0.12193, accuracy:0.98000
epoch:1200, cost:0.11761, accuracy:0.98667
epoch:1300, cost:0.11385, accuracy:0.98667
epoch:1400, cost:0.11053, accuracy:0.98667
epoch:1500, cost:0.10759, accuracy:0.98667
epoch:1600, cost:0.10495, accuracy:0.98667
epoch:1700, cost:0.10257, accuracy:0.98667
epoch:1800, cost:0.10042, accuracy:0.98667
epoch:1900, cost:0.09846, accuracy:0.98667
epoch:2000, cost:0.09666, accuracy:0.98667


In [58]:
predict=model(x_train).argmax(dim=1)
print(predict)
(predict==y_train).sum()/len(predict)*100

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])


tensor(98.6667)